In [1]:
import pandas as pd
import numpy as np
import h5py
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn.client import Client
import matplotlib.pyplot as plt

In [2]:
def make_stream(dataset,trace='eq_trace'):
    '''
    input: hdf5 dataset
    output: obspy stream
   
    '''
    data = np.array(dataset)
      
    '''for x-axis'''
    tr_E = obspy.Trace(data=data[:, 0]) #passing x data
    if (trace == 'eq_trace'):
        tr_E.stats.starttime = UTCDateTime(dataset.attrs['trace_start_time'])
    else:
        tr_E.stats.starttime = UTCDateTime(trace.split('_')[1]) # for noise data
    tr_E.stats.delta = 0.01
    tr_E.stats.channel = dataset.attrs['receiver_type']+'E'
    tr_E.stats.station = dataset.attrs['receiver_code']
    tr_E.stats.network = dataset.attrs['network_code']
   
    #for y-axis
    tr_N = obspy.Trace(data=data[:, 1]) #passing y data
    if (trace == 'eq_trace'):
        tr_N.stats.starttime = UTCDateTime(dataset.attrs['trace_start_time'])
    else:
        tr_N.stats.starttime = UTCDateTime(trace.split('_')[1])  # for noise data
    tr_N.stats.delta = 0.01
    tr_N.stats.channel = dataset.attrs['receiver_type']+'N'
    tr_N.stats.station = dataset.attrs['receiver_code']
    tr_N.stats.network = dataset.attrs['network_code']
   
    #for z-axis
    tr_Z = obspy.Trace(data=data[:, 2])
    if (trace == 'eq_trace'):
        tr_Z.stats.starttime = UTCDateTime(dataset.attrs['trace_start_time'])
    else:
        tr_Z.stats.starttime = UTCDateTime(trace.split('_')[1])  # for noise data
    tr_Z.stats.delta = 0.01
    tr_Z.stats.channel = dataset.attrs['receiver_type']+'Z'
    tr_Z.stats.station = dataset.attrs['receiver_code']
    tr_Z.stats.network = dataset.attrs['network_code']

    stream = obspy.Stream([tr_E, tr_N, tr_Z])
   
    return stream

# Fetching Earthquake Waveforms

In [3]:
eq_meta = pd.read_csv('metadata//eq_meta.csv')

In [4]:
eq_meta[['source_magnitude']]

,source_magnitude
0,6.30
1,6.20
2,0.87
3,0.70
4,0.90
...,...
197,5.20
198,5.10
199,7.90
200,7.70


In [211]:
eq_meta.dtypes

network_code                         object
receiver_code                        object
receiver_type                        object
receiver_latitude                   float64
receiver_longitude                  float64
receiver_elevation_m                float64
p_arrival_sample                    float64
p_status                             object
p_weight                            float64
p_travel_sec                        float64
s_arrival_sample                    float64
s_status                             object
s_weight                            float64
source_id                            object
source_origin_time                   object
source_origin_uncertainty_sec        object
source_latitude                     float64
source_longitude                    float64
source_error_sec                     object
source_gap_deg                       object
source_horizontal_uncertainty_km     object
source_depth_km                     float64
source_depth_uncertainty_km     

In [212]:
eq_meta.shape

(200, 35)

In [10]:
#hdf points to waveforms.hdf5
hdf = h5py.File('../waveforms.hdf5')

#displaying groups and sub groups of the file
for group in hdf:
    for sub_group in hdf[group]:
        print('Group: ' + group)
        print('Sub-group: ' + sub_group)
        print("====")

Group: earthquake
Sub-group: local
====
Group: non_earthquake
Sub-group: noise
====


In [6]:
#making a list of traces for earthquake
traceList = eq_meta['trace_name'].to_list()

In [7]:
#making eq_waveform data frame that consists of earthquake waveforms
eq_waveforms_counts = pd.DataFrame(columns = ['x', 'y', 'z'])
eq_waveforms_acc = pd.DataFrame(columns = ['x', 'y', 'z'])

for trace_name in traceList:
    dataset = hdf.get('/earthquake/local/'+str(trace_name))
    
    '''
    for acceleration values
    '''
    # convering hdf5 dataset into obspy sream
    # downloading the instrument response of the station from IRIS
    client = Client("IRIS")
    inventory = client.get_stations(network=dataset.attrs['network_code'],
                                station=dataset.attrs['receiver_code'],
                                starttime=UTCDateTime(dataset.attrs['trace_start_time']),
                                endtime=UTCDateTime(dataset.attrs['trace_start_time']) + 60,
                                loc="*",
                                channel="*",
                                level="response")  
            
    # converting into acceleration
    st = make_stream(dataset)
    try:
        st.remove_response(inventory=inventory, output="ACC", plot=False)
    except:
        continue
    temp = pd.DataFrame()
    temp['x'] = st[0].data
    temp['y'] = st[1].data
    temp['z'] = st[2].data
    eq_waveforms_acc = eq_waveforms_acc.append(temp, ignore_index = True)
    
    '''
    for amplitude counts
    '''
    dataset = np.array(dataset)
    temp = pd.DataFrame(columns=['x', 'y', 'z'], data = dataset)
    eq_waveforms_counts = eq_waveforms_counts.append(temp, ignore_index = True)

C:\Users\sohai\Anaconda3\lib\site-packages\obspy\io\stationxml\core.py:84: UserWarning: The StationXML file has version 1.1, ObsPy can deal with version 1.0. Proceed with caution.
  root.attrib["schemaVersion"], SCHEMA_VERSION))


### We end up getting 200 earthquakes. Couldn't find response for 2 earthquakes

In [8]:
eq_waveforms_acc.shape

(1200000, 3)

In [9]:
eq_waveforms_counts.shape

(1200000, 3)

In [10]:
#Saves waveforms to csv files
eq_waveforms_acc.to_csv('waveforms//eq_waveforms_acc.csv', index=False)
eq_waveforms_counts.to_csv('waveforms//eq_waveforms_counts.csv', index=False)

# Fetching Noise Waveforms

In [2]:
noise_meta = pd.read_csv('metadata//noise_meta.csv')

In [3]:
noise_meta.head()

,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
0,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,noise,109C.TA_201510210555_NO
1,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,noise,109C.TA_201601150401_NO
2,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,noise,109C.TA_201605040518_NO
3,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,noise,109C.TA_201607130559_NO
4,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,noise,109C.TA_201610291226_NO


In [5]:
noise_meta.shape

(200, 35)

In [6]:
#making a list of traces for noise
traceList = noise_meta['trace_name'].to_list()

In [7]:
traceList

['109C.TA_201510210555_NO',
 '109C.TA_201601150401_NO',
 '109C.TA_201605040518_NO',
 '109C.TA_201607130559_NO',
 '109C.TA_201610291226_NO',
 '109C.TA_201701032247_NO',
 '109C.TA_201703300645_NO',
 '109C.TA_201705251906_NO',
 '109C.TA_201707201037_NO',
 '109C.TA_201709141044_NO',
 '109C.TA_201710011403_NO',
 '109C.TA_201711141526_NO',
 '109C.TA_201801160903_NO',
 '109C.TA_201803291025_NO',
 '109C.TA_201807071249_NO',
 '113A.AE_201503301529_NO',
 '113A.AE_201504251324_NO',
 '113A.AE_201505080822_NO',
 '113A.AE_201505211651_NO',
 '113A.AE_201505261458_NO',
 '113A.AE_201506121638_NO',
 '113A.AE_201507101131_NO',
 '113A.AE_201508091108_NO',
 '113A.AE_201509040039_NO',
 '113A.AE_201509171056_NO',
 '113A.AE_201510271134_NO',
 '113A.AE_201511151922_NO',
 '113A.AE_201511271712_NO',
 '113A.AE_201512101925_NO',
 '113A.AE_201601021938_NO',
 '113A.AE_201601212222_NO',
 '113A.AE_201602090846_NO',
 '113A.AE_201602240656_NO',
 '113A.AE_201603111359_NO',
 '113A.AE_201603271144_NO',
 '113A.AE_2016040915

In [19]:
#making noise_waveform data frame that consists of noise waveforms
noise_waveforms_counts = pd.DataFrame(columns = ['x', 'y', 'z'])
noise_waveforms_acc = pd.DataFrame(columns = ['x', 'y', 'z'])

for trace_name in traceList:
    dataset = hdf.get('/non_earthquake/noise/'+str(trace_name))
    
    '''
    for acceleration values
    '''
    # convering hdf5 dataset into obspy sream
    st = make_stream(dataset, trace_name)
    # downloading the instrument response of the station from IRIS
    client = Client("IRIS")
    inventory = client.get_stations(network=dataset.attrs['network_code'],
                                station=dataset.attrs['receiver_code'],
                                starttime=UTCDateTime(trace_name.split('_')[1]),
                                endtime=UTCDateTime(trace_name.split('_')[1]) + 60,
                                loc="*",
                                channel="*",
                                level="response")  
            
    # converting into acceleration
    st = make_stream(dataset,trace_name)
    try:
        st.remove_response(inventory=inventory, output="ACC", plot=False)
    except:
        continue
    temp = pd.DataFrame()
    #st is a nested list with each list corresponding to a different axis 
    temp['x'] = st[0].data
    temp['y'] = st[1].data
    temp['z'] = st[2].data
    noise_waveforms_acc = noise_waveforms_acc.append(temp, ignore_index = True)
    
    '''
    for amplitude counts
    '''
    dataset = np.array(dataset)
    temp = pd.DataFrame(columns=['x', 'y', 'z'], data = dataset)
    noise_waveforms_counts = noise_waveforms_counts.append(temp, ignore_index = True)

In [20]:
noise_waveforms_acc.shape

(1200000, 3)

In [21]:
noise_waveforms_acc.head()

,x,y,z
0,1.182854e-09,1.182854e-09,1.182854e-09
1,2.883696e-10,2.883696e-10,2.883696e-10
2,-1.036030e-09,-1.036030e-09,-1.036030e-09
3,-2.001298e-09,-2.001298e-09,-2.001298e-09
4,-2.127769e-09,-2.127769e-09,-2.127769e-09


In [22]:
noise_waveforms_counts.head()

,x,y,z
0,-0.000000,-0.000000,-0.000000
1,-0.004858,-0.004858,-0.004858
2,-0.027905,-0.027905,-0.027905
3,-0.082969,-0.082969,-0.082969
4,-0.121807,-0.121807,-0.121807


In [23]:
#Saves waveforms to csv files
noise_waveforms_acc.to_csv('waveforms//noise_waveforms_acc.csv', index=False)
noise_waveforms_counts.to_csv('waveforms//noise_waveforms_counts.csv', index=False)